In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_stats
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy

In [2]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [3]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle(DATA_DIR+'bureau_data.pkl')

In [4]:
train.shape,test.shape,bur_df.shape

((128655, 26), (14745, 25), (591000, 32))

In [5]:
########### WRONG SIMULATION DATE CONVERSION

# train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
# test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
# train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
# train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])

train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
test['MaturityDAte'] = pd.to_datetime(test['MaturityDAte'])


bur_df['DATE-REPORTED'] = pd.to_datetime(bur_df['DATE-REPORTED'])
bur_df['dpd_strin_var'] = bur_df['dpd_string'].fillna('').apply(lambda x: [x[y-3:y] for y in range(3, len(x)+3, 3)] if x.find('E')==-1 else ['000'])

In [6]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [7]:
df.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10 00:00:00,4022465,1568.0,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31 00:00:00,4681175,1062.0,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17 00:00:00,25328146,1060.0,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29 00:00:00,13021591,1060.0,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08 00:00:00,3291320,1046.0,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


#### create features for the period of LTFS loan present in the application data, like time to next loan, time to last loan,CIBIL fc on the tradelines originated in the period

In [8]:
bur_df = bur_df.merge(df[['ID','DisbursalDate','MaturityDAte','DisbursalAmount']],on='ID')
bur_df.sort_values(['ID','DISBURSED-DT'],inplace=True)

In [9]:
(bur_df['SELF-INDICATOR']==True).value_counts(1)

False    0.694228
True     0.305772
Name: SELF-INDICATOR, dtype: float64

In [10]:
bur_df.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,min_reported_date,max_reported_date,dpd_str_len,dpd_string,correctedOVERDUE-AMT,correctedDISBURSED-AMT/HIGH CREDIT,correctedCURRENT-BAL,dpd_strin_var,DisbursalDate,MaturityDAte,DisbursalAmount
8,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2013-03-31,2016-02-29,108,0000000000000000000000000000000000000000000000...,0.0,275000.0,0.0,"[000, 000, 000, 000, 000, 000, 000, 000, 000, ...",2012-02-10,2016-01-15,275000.0
6,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-01-30,2017-09-21,NaT,NaN,"5,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0000000000000870540270270000320000000000...,"0,,0,0,0,0,0,0,0,13060,13060,25982,38767,51417...","0,,0,0,0,0,3064,3064,3064,2972,0,2802,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-10-31,2017-09-30,108,000DDD0000000000000870540270270000320000000000...,0.0,500000.0,0.0,"[000, DDD, 000, 000, 000, 000, 087, 054, 027, ...",2012-02-10,2016-01-15,275000.0
7,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-07-31,Individual,Closed,2013-06-11,2017-07-01,NaT,NaN,"4,00,000",NaN,0,NaN,0,0.0,NaN,"20170731,20170601,20170531,20170430,20170331,2...",000DDD0220210220190220220000270000000000000000...,"0,,0,10487,20862,31126,41281,51328,51328,71101...","0,,1722,1722,1722,1722,1722,1722,0,1625,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-08-31,2017-07-31,108,000DDD0220210220190220220000270000000000000000...,0.0,400000.0,0.0,"[000, DDD, 022, 021, 022, 019, 022, 022, 000, ...",2012-02-10,2016-01-15,275000.0
5,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN,2014-10-31,2017-09-30,108,000DDD0270260270260270240270270000320000000000...,0.0,300000.0,0.0,"[000, DDD, 027, 026, 027, 026, 027, 024, 027, ...",2012-02-10,2016-01-15,275000.0
3,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaT,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,2018-03-31,2018-04-30,6,030000,37873.0,37352.0,37873.0,"[030, 000]",2012-02-10,2016-01-15,275000.0


In [11]:
def get_bureau_feats_2(grp):
    _id,account_df = grp
    account_df['disb_ratio'] = account_df['correctedDISBURSED-AMT/HIGH CREDIT']/account_df['DisbursalAmount']
    _dict = {}
    _dict["individual_accounts"] = (account_df["OWNERSHIP-IND"] == "Individual").sum()
    _dict["joint_accounts"] = (account_df["OWNERSHIP-IND"] == "Joint").sum()
    _dict["guarantor_accounts"] = (account_df["OWNERSHIP-IND"] == "Guarantor").sum()
    _dict["curr_bal_grtr_0"] = (account_df["correctedCURRENT-BAL"] > 0).sum()
    _dict["num_accounts"] = len(account_df)
    _dict.update(
        get_stats(
            account_df["correctedDISBURSED-AMT/HIGH CREDIT"],
            "correctedDISBURSED-AMT/HIGH CREDIT",
        )
    )
    _dict.update(get_stats(account_df["correctedCURRENT-BAL"], "correctedCURRENT-BAL"))
    _dict.update(get_stats(account_df["correctedOVERDUE-AMT"], "correctedOVERDUE-AMT"))
    _dict["num_closed_accounts"] = (account_df["ACCOUNT-STATUS"] == "Closed").sum()
    _dict["num_open_accounts"] = (account_df["ACCOUNT-STATUS"] == "Active").sum()
    _dict["num_delinq_accounts"] = (account_df["ACCOUNT-STATUS"] == "Delinquent").sum()
    _dict["total_written_off_amount"] = account_df["WRITE-OFF-AMT"].sum()
    perc_paid_off = (
        1
        - account_df["correctedCURRENT-BAL"]
        / account_df["correctedDISBURSED-AMT/HIGH CREDIT"]
    )
    _dict.update(get_stats(perc_paid_off, "percent_paid_off", include_sum=False))
    _dict.update(get_stats(account_df['disb_ratio'], "disb_ratio"))    
    
    _dict["overall_percent_paid_off"] = (
        1
        - account_df["correctedCURRENT-BAL"].sum()
        / account_df["correctedDISBURSED-AMT/HIGH CREDIT"].sum()
    )
    tenors = ((account_df["DATE-REPORTED"] - account_df["DISBURSED-DT"]).dt.days).clip(
        upper=7300
    )
    _dict["median_tenor"] = tenors.median()
    _dict["max_tenor"] = tenors.max()
    _dict["min_tenor"] = tenors.min()

#     loan_ids = [
#         "Tractor Loan",
#         "Gold Loan",
#         "Overdraft"
#     ]
#     for loan in loan_ids:
#         _filter = account_df["ACCT-TYPE"] == loan
#         temp = account_df[_filter]
#         _dict["num_accounts_{}".format(loan)] = len(temp)
#         _dict["total_sanctioned_amount_{}".format(loan)] = temp[
#             "correctedDISBURSED-AMT/HIGH CREDIT"
#         ].sum()
#         _dict["total_curr_bal_{}".format(loan)] = temp["correctedCURRENT-BAL"].sum()
#         _dict["overall_percentage_paid_off_{}".format(loan)] = (
#             1
#             - temp["correctedCURRENT-BAL"].sum()
#             / temp["correctedDISBURSED-AMT/HIGH CREDIT"].sum()
#         )

    days_diff = (account_df["DISBURSED-DT"] - account_df["DisbursalDate"]).dt.days
    _dict.update(
        get_stats(
            days_diff[days_diff > 0],
            "day_start_day_diff_app_vs_other",
            include_sum=False,
        )
    )
    
    days_diff = (account_df["MaturityDAte"]-account_df["DISBURSED-DT"]).dt.days
    _dict.update(
        get_stats(
            days_diff[days_diff > 0],
            "time_delta_anchor_close_vs_start",
            include_sum=False,
        )
    )
    _dict['num_ltfs_loans'] = account_df['SELF-INDICATOR'].sum()
    
#     _dict.update(
#         get_stats(
#             (account_df["DISBURSED-DT"] - account_df["DISBURSED-DT"].shift(1)).dt.days,
#             "days_bw_loans",
#         )
#     )
    _dict["ID"] = _id
    return _dict

### features for the accounts within 2 years of disrbursal of current loan

In [12]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*2)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats1 = pd.DataFrame(data_outputs)    

47191 (95480, 36)
CPU times: user 21 s, sys: 713 ms, total: 21.7 s
Wall time: 1min


In [13]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>(temp_df.DisbursalDate+dt.timedelta(days=365*2)))&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*4)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats2 = pd.DataFrame(data_outputs)    

43875 (89145, 36)
CPU times: user 20.3 s, sys: 793 ms, total: 21.1 s
Wall time: 57.7 s


In [14]:
%%time
temp_df = copy.deepcopy(bur_df)
cond1 = (temp_df['DISBURSED-DT']>(temp_df.DisbursalDate+dt.timedelta(days=365*4)))&(temp_df['DISBURSED-DT']<=(temp_df.DisbursalDate+dt.timedelta(days=365*10)))
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)
temp_df = temp_df[cond1]
print(temp_df.ID.nunique(),temp_df.shape)
grps = temp_df.groupby('ID')

try:
    pool = Pool(8) 
    data_outputs = pool.map(get_bureau_feats_2, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()
    
bur_feats3 = pd.DataFrame(data_outputs)    

32304 (88485, 36)
CPU times: user 14.9 s, sys: 773 ms, total: 15.7 s
Wall time: 41.3 s


In [15]:
bur_feats1.shape,bur_feats2.shape,bur_feats3.shape

((47191, 47), (43875, 47), (32304, 47))

In [16]:
bur_feats1.columns = ['2_yr_frm_disb'+x if x!='ID' else x for x in bur_feats1.columns]
bur_feats2.columns = ['2-4_yr_frm_disb'+x if x!='ID' else x for x in bur_feats2.columns]
bur_feats3.columns = ['grtr_4_yr_frm_disb'+x if x!='ID' else x for x in bur_feats3.columns]

In [ ]:
df[['ID']].merge(bur_feats1,on='ID',how='left').merge(bur_feats2,on='ID',how='left').merge(bur_feats3,on='ID',how='left').fillna(0).to_pickle('future_from_disb_date_feats.pkl')

In [8]:
temp_df = copy.deepcopy(bur_df)
temp_df.MaturityDAte = temp_df.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=temp_df.MaturityDAte)
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)

temp_df = temp_df[cond1]

(139964, (362624, 35))

In [10]:
# bur_df.ID.nunique()

In [11]:
# grps = temp_df[temp_df.ID.isin(train.ID.sample(1000))].groupby('ID')
grps = temp_df.groupby('ID')

In [16]:
def get_post_feats(grp):
    id_,temp = grp
    temp.MaturityDAte = temp.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
    cond1 = (temp['DISBURSED-DT']>=temp.DisbursalDate)&(temp['DISBURSED-DT']<=temp.MaturityDAte)
    account_df = temp[cond1]
#     account_df = temp
    return get_bureau_feats(account_df)

In [12]:
# temp_df.ID.value_counts()

In [13]:
# len(np.array([]))

In [14]:
# for g in grps:
#     id_,temp = g
#     temp.MaturityDAte = temp.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
#     cond1 = (temp['DISBURSED-DT']>=temp.DisbursalDate)&(temp['DISBURSED-DT']<=temp.MaturityDAte)
#     account_df = temp[cond1]
#     get_bureau_feats(account_df)

In [15]:
# temp

In [16]:
# temp.DisbursalDate

In [13]:
%%time
try:
    pool = Pool(8) 
    data_outputs = pool.map(get_post_feats, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

CPU times: user 1min 13s, sys: 5 s, total: 1min 18s
Wall time: 8min 48s


In [14]:
bur_feats = pd.DataFrame(data_outputs)

In [15]:
bur_feats.head()

,individual_accounts,joint_accounts,guarantor_accounts,curr_bal_grtr_0,num_accounts,mean_correctedDISBURSED-AMT/HIGH CREDIT,min_correctedDISBURSED-AMT/HIGH CREDIT,max_correctedDISBURSED-AMT/HIGH CREDIT,std_correctedDISBURSED-AMT/HIGH CREDIT,sum_correctedDISBURSED-AMT/HIGH CREDIT,mean_correctedCURRENT-BAL,min_correctedCURRENT-BAL,max_correctedCURRENT-BAL,std_correctedCURRENT-BAL,sum_correctedCURRENT-BAL,mean_correctedOVERDUE-AMT,min_correctedOVERDUE-AMT,max_correctedOVERDUE-AMT,std_correctedOVERDUE-AMT,sum_correctedOVERDUE-AMT,num_closed_accounts,num_open_accounts,num_delinq_accounts,total_written_off_amount,mean_percent_paid_off,min_percent_paid_off,max_percent_paid_off,std_percent_paid_off,overall_percent_paid_off,mean_emi_proxy,min_emi_proxy,max_emi_proxy,std_emi_proxy,sum_emi_proxy,mean_payment_measure_1,min_payment_measure_1,max_payment_measure_1,std_payment_measure_1,sum_payment_measure_1,median_tenor,max_tenor,min_tenor,std_count,ddd_count,xxx_count,late_count,_30_count,_60_count,_90_count,_180_count,total_count,std_count_application_loan,ddd_count_application_loan,xxx_count_application_loan,late_count_application_loan,_30_count_application_loan,_60_count_application_loan,_90_count_application_loan,_180_count_application_loan,total_count_application_loan,mean_dpd_str,min_dpd_str,max_dpd_str,std_dpd_str,mean_dpd_str_application_loan,min_dpd_str_application_loan,max_dpd_str_application_loan,std_dpd_str_application_loan,num_accounts__between_0_and_365_days,median_tenor__between_0_and_365_days,mean_emi_proxy__between_0_and_365_days,min_emi_proxy__between_0_and_365_days,max_emi_proxy__between_0_and_365_days,std_emi_proxy__between_0_and_365_days,sum_emi_proxy__between_0_and_365_days,mean_perc_paid_off__between_0_and_365_days,min_perc_paid_off__between_0_and_365_days,max_perc_paid_off__between_0_and_365_days,std_perc_paid_off__between_0_and_365_days,sum_perc_paid_off__between_0_and_365_days,mean_payment_measure_1__between_0_and_365_days,min_payment_measure_1__between_0_and_365_days,max_payment_measure_1__between_0_and_365_days,std_payment_measure_1__between_0_and_365_days,sum_payment_measure_1__between_0_and_365_days,mean_correctedDISBURSED-AMT/HIGH CREDIT__between_0_and_365_days,min_correctedDISBURSED-AMT/HIGH CREDIT__between_0_and_365_days,max_correctedDISBURSED-AMT/HIGH CREDIT__between_0_and_365_days,std_correctedDISBURSED-AMT/HIGH CREDIT__between_0_and_365_days,sum_correctedDISBURSED-AMT/HIGH CREDIT__between_0_and_365_days,mean_correctedCURRENT-BAL__between_0_and_365_days,min_correctedCURRENT-BAL__between_0_and_365_days,max_correctedCURRENT-BAL__between_0_and_365_days,std_correctedCURRENT-BAL__between_0_and_365_days,sum_correctedCURRENT-BAL__between_0_and_365_days,num_accounts__between_365_and_730_days,median_tenor__between_365_and_730_days,mean_emi_proxy__between_365_and_730_days,min_emi_proxy__between_365_and_730_days,max_emi_proxy__between_365_and_730_days,std_emi_proxy__between_365_and_730_days,sum_emi_proxy__between_365_and_730_days,mean_perc_paid_off__between_365_and_730_days,min_perc_paid_off__between_365_and_730_days,max_perc_paid_off__between_365_and_730_days,std_perc_paid_off__between_365_and_730_days,sum_perc_paid_off__between_365_and_730_days,mean_payment_measure_1__between_365_and_730_days,min_payment_measure_1__between_365_and_730_days,max_payment_measure_1__between_365_and_730_days,std_payment_measure_1__between_365_and_730_days,sum_payment_measure_1__between_365_and_730_days,mean_correctedDISBURSED-AMT/HIGH CREDIT__between_365_and_730_days,min_correctedDISBURSED-AMT/HIGH CREDIT__between_365_and_730_days,max_correctedDISBURSED-AMT/HIGH CREDIT__between_365_and_730_days,std_correctedDISBURSED-AMT/HIGH CREDIT__between_365_and_730_days,sum_correctedDISBURSED-AMT/HIGH CREDIT__between_365_and_730_days,mean_correctedCURRENT-BAL__between_365_and_730_days,min_correctedCURRENT-BAL__between_365_and_730_days,max_correctedCURRENT-BAL__between_365_and_730_days,std_correctedCURRENT-BAL__between_365_and_730_days,sum_correcte

In [16]:
bur_feats.shape

(139964, 213)

In [17]:
bur_feats.to_pickle('bureau_future_feats_v2.pkl')

#### ltfs loan variables

In [12]:
temp_df = copy.deepcopy(bur_df)
temp_df.MaturityDAte = temp_df.MaturityDAte+ dt.timedelta(days=730) ### loans post 2 years of completetion of the current one
cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)&(temp_df['DISBURSED-DT']<=temp_df.MaturityDAte)
cond2 = bur_df['SELF-INDICATOR']==True
# cond1 = (temp_df['DISBURSED-DT']>=temp_df.DisbursalDate)

temp_df = temp_df[cond1&cond2]

In [13]:
temp_df.ID.nunique(),temp_df.shape

(137534, (166712, 35))

In [20]:
# grps = temp_df[temp_df.ID.isin(train.ID.sample(1000))].groupby('ID')
grps = temp_df.groupby('ID')

In [21]:
%%time
try:
    pool = Pool(8) 
    data_outputs = pool.map(get_post_feats, grps)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

CPU times: user 1min 8s, sys: 2.37 s, total: 1min 11s
Wall time: 8min 44s


In [22]:
ltfs_bur_feats = pd.DataFrame(data_outputs)
ltfs_bur_feats.shape

(137534, 120)

In [3]:
ltfs_bur_feats = pd.read_pickle('ltfs_bureau_future_feats.pkl')

In [8]:
ltfs_bur_feats.columns = ['ltfs_'+x if x!='ID' else x for x in ltfs_bur_feats.columns]

In [9]:
ltfs_bur_feats.to_pickle('ltfs_bureau_future_feats.pkl')